440. K-th Smallest in Lexicographical Order
     Solved
     Hard
     Topics
     premium lock icon
     Companies
     Given two integers n and k, return the kth lexicographically smallest integer in the range [1, n].

Example 1:

Input: n = 13, k = 2
Output: 10
Explanation: The lexicographical order is [1, 10, 11, 12, 13, 2, 3, 4, 5, 6, 7, 8, 9], so the second smallest number is 10.
Example 2:

Input: n = 1, k = 1
Output: 1

Constraints:

1 <= k <= n <= 109


The problem "K-th Smallest in Lexicographical Order" asks us to find the $k$-th lexicographically smallest integer within the range $[1, n]$. The constraints are significant: $1 \le k \le n \le 10^9$. This large value of $n$ means that we cannot generate all numbers and sort them (as that would be $O(n)$ time and $O(n)$ space, too slow for $10^9$). We need a more efficient approach, likely logarithmic in $n$.

### Understanding Lexicographical Order

Lexicographical order treats numbers like strings.

- `1` comes before `2`.
- `10` comes before `2` because '1' < '2'.
- `100` comes before `11` because '100' > '11' (lexicographically, it's '1' vs '1' then '0' vs '1', so '100' is less than '11' if it's based on prefix, but here it's full string comparison: '100' is greater than '11' as '0' is greater than '1' when comparing third char). No, this is wrong.
  - `1`
  - `10`
  - `100`
  - `101`
  - ...
  - `11`
  - `110`
  - ...
  - `12`
  - ...
  - `2`
  - `20`

This order is essentially a pre-order traversal of a 10-ary (decimal) tree where each node represents a number.

```
          (Root - not a number)
          / | \ ... \
         1  2   3   ... 9
        /|\ /|\ ... /|\
       10 11 .. 19  20 21 .. 29 ...
      /|
     100 101 ...
```

Our goal is to find the $k$-th node visited in this pre-order traversal.

---

### Approach 1: Brute Force (Not Feasible for $n=10^9$)

**Algorithm:**

1.  Generate all numbers from 1 to $n$.
2.  Convert them to strings.
3.  Sort the strings lexicographically.
4.  Return the integer representation of the string at index $k-1$.

**Time Complexity:** $O(N \log N \cdot \log_{10}N)$ due to sorting $N$ strings of average length $\log_{10}N$. For $N=10^9$, this is far too slow.
**Space Complexity:** $O(N \cdot \log_{10}N)$ for storing all strings. Too much for $N=10^9$.

---

### Approach 2: Depth-First Search (Recursive) (Not Feasible for $n=10^9$)

This approach directly simulates the tree traversal, but it's still too slow if $k$ is large, as it might have to visit many nodes.

**Algorithm:**

1.  Initialize an empty list `result`.
2.  Define a recursive `dfs(current_num)` function:
    - If `current_num > n`, return.
    - Add `current_num` to `result`.
    - For `digit` from 0 to 9:
      - `next_num = current_num * 10 + digit`.
      - If `next_num > n`, break (no need to check larger digits).
      - Recursively call `dfs(next_num)`.
3.  Call `dfs(i)` for `i` from 1 to 9 (if `i <= n`).
4.  Return `result[k-1]`.

**Time Complexity:** In the worst case (e.g., $k$ is close to $N$), it still visits almost all $N$ numbers, leading to $O(N)$ time. This is too slow for $N=10^9$.
**Space Complexity:** $O(N)$ for the `result` list and $O(\log_{10}N)$ for recursion depth.

---

### Approach 3: Optimized Iterative Traversal / Counting Subtree Sizes (The Efficient Solution)

This is the standard and most efficient approach for this problem. The core idea is to efficiently count how many numbers are lexicographically smaller than a given prefix/number, allowing us to "jump" through the tree without visiting every node.

**Key Idea:**
We start at `curr = 1` and we have `k-1` steps to take (since $k$ is 1-indexed and we are looking for the $k$-th element, and `curr=1` is already the 1st element).
In each step, we decide whether to go "down" (append a 0) or "right" (increment the current number). We make this decision by calculating how many numbers are in the "subtree" rooted at `curr * 10`.

Let's define a helper function `get_count(n, prefix)` that calculates the total count of numbers starting with `prefix` (or being `prefix` itself) that are less than or equal to `n`.

`get_count(n, prefix)`:
This function essentially counts all numbers in the subtree starting with `prefix` up to `n`.

- `count = 0`
- `curr = prefix`
- `next = prefix + 1` (the next prefix at the same level)
- While `curr <= n`:
  - `count += min(n + 1, next) - curr` (This adds the number of elements in the range `[curr, min(n, next-1)]`.
    - For example, if `prefix = 1`, `next = 2`. The range is `[1, 1]`. Count is `1`.
    - If `prefix = 10`, `next = 11`. The range is `[10, 10]`. Count is `1`.
    - If `prefix = 1` and `n = 13`:
      - `curr = 1`, `next = 2`. `count += min(14, 2) - 1 = 2 - 1 = 1`. (for 1)
      - `curr = 10`, `next = 20`. `count += min(14, 20) - 10 = 14 - 10 = 4`. (for 10, 11, 12, 13)
      - `curr = 100`, `next = 200`. `100 > 13`, loop terminates.
    - This sum actually represents `min(n, next_level_start - 1) - current_level_start + 1`. This is simpler: just count numbers between `curr` and `next-1` that are `<=n`.
  - `curr *= 10` (go to the next level down for `prefix * 10`)
  - `next *= 10` (go to the next level down for `(prefix+1) * 10`)
- Return `count`.

**Algorithm Steps:**

1.  Initialize `current_num = 1`. This is our current candidate for the $k$-th number.
2.  Decrement `k` by 1 (`k -= 1`). This is because we start counting from `1`, and `current_num = 1` is already the first lexicographical number. So we need to find the $(k-1)$-th number _after_ `1`.
3.  Loop while `k > 0`:

    - Calculate `steps = get_count(n, current_num * 10)`. This `steps` variable tells us how many numbers are in the subtree rooted at `current_num * 10` (i.e., all numbers starting with `current_num` followed by a digit).
    - **Decision:**
      - If `k >= steps`: This means the $k$-th number is NOT in the subtree rooted at `current_num * 10`. It must be lexicographically larger, so we need to move to the "right" sibling.
        - `current_num += 1`
        - `k -= steps` (we've skipped all numbers in the subtree)
        - `k -= 1` (we've also skipped the `current_num` itself at this level when going right)
      - Else (`k < steps`): This means the $k$-th number IS in the subtree rooted at `current_num * 10`. We need to "go down" into this subtree.
        - `current_num *= 10`
        - `k -= 1` (we've consumed one step by going down to `current_num * 10`).

4.  Return `current_num`.

**Detailed `get_count` function:**

```python
def get_count(n: int, prefix: int) -> int:
    """
    Counts how many numbers in the range [1, n] start with `prefix`
    (or are exactly `prefix` if prefix <= n).
    """
    count = 0
    # `cur` represents the starting number of the current level
    # `next_val` represents the starting number of the next sibling prefix
    #             at the current level (e.g., if cur=1, next_val=2)
    #             or the next level (e.g., if cur=10, next_val=11)
    cur = prefix
    next_val = prefix + 1

    while cur <= n:
        # Calculate numbers in the range [cur, min(n, next_val - 1)]
        # This handles numbers at the current 'depth' that start with 'cur'
        count += min(n + 1, next_val) - cur

        # Move to the next level (multiply by 10)
        # For example, from 1 to 10, then 100, then 1000 etc.
        cur *= 10
        next_val *= 10
    return count
```

**Trace `n = 13, k = 2`:**

1.  `current_num = 1`, `k = 1` (since $k$-th, and we are starting from 1)
2.  Loop while `k > 0`:
    - `current_num = 1`.
    - `steps = get_count(13, 1 * 10) = get_count(13, 10)`
      - `count = 0`, `cur = 10`, `next_val = 11`
      - `cur = 10 <= 13`:
        - `count += min(13+1, 11) - 10 = min(14, 11) - 10 = 11 - 10 = 1`. (`count = 1` for `10`)
        - `cur = 100`, `next_val = 110`
      - `cur = 100 > 13`: Loop ends.
      - `get_count(13, 10)` returns `1` (only `10` itself if $n=10$, but if $n=13$, it includes 10, 11, 12, 13, so the logic is slightly off. Let's re-verify `get_count`).

**Revised `get_count` logic (and why it works):**
When `prefix` is `P`, we are counting all numbers in the form `P`, `P0`, `P00`, ..., `P9`, `P01`, ..., `P99`, etc.
The crucial part is that `next_val` is `(prefix + 1)` which essentially represents the start of the "next sibling's subtree" at the same level of prefix.

Example: `get_count(13, 1)`
`prefix = 1`, `cur = 1`, `next_val = 2`

- `cur=1 <= 13`: `count += min(14, 2) - 1 = 1`. (This counts `1`)
  - `cur = 10`, `next_val = 20`
- `cur=10 <= 13`: `count += min(14, 20) - 10 = 4`. (This counts `10, 11, 12, 13`)
  - `cur = 100`, `next_val = 200`
- `cur=100 > 13`: Loop ends.
  Total `count = 1 + 4 = 5`.
  This `get_count(13, 1)` correctly counts `1, 10, 11, 12, 13`.

Now, back to the main loop:

**Trace `n = 13, k = 2`:**

1.  `current_num = 1`, `k = 1` (target: 2nd element, 1st is `1`, so 1 step left)
2.  Loop while `k > 0`:
    - `current_num = 1`.
    - `steps = get_count(13, 1 * 10) = get_count(13, 10)`:
      - `count = 0`, `cur = 10`, `next_val = 11`
      - `cur = 10 <= 13`: `count += min(14, 11) - 10 = 1`. (Counts `10`)
        - `cur = 100`, `next_val = 110`
      - `cur = 100 > 13`: Loop ends.
      - `get_count(13, 10)` returns `1`.
    - Is `k >= steps`? (`1 >= 1` is True)
      - Go right: `current_num = 1 + 1 = 2`
      - `k -= steps` (`k = 1 - 1 = 0`)
      - `k -= 1` (`k = 0 - 1 = -1`)
    - Loop condition `k > 0` (is `-1 > 0`?) is False. Loop terminates.
3.  Return `current_num = 2`.

This is incorrect for `n=13, k=2`. The output should be 10. The problem is in the `k -= 1` step when moving right. When you go right from `current_num`, you are effectively skipping `current_num` and all of its descendants. The `get_count` already counts `current_num` and its descendants.

The logic should be: if we are at `current_num`, we want to find the $k$-th element.
We compare `k` with `count(current_num * 10)`.

Let `k` be 1-indexed. `ans = 1`.

Revised Algorithm:

1.  Initialize `ans = 1`. (This will be our $k$-th number)
2.  Decrement `k` by 1 (`k -= 1`). (Since we start finding from `1`, which is the 1st element)
3.  Loop while `k > 0`:

    - Calculate `count_nodes_in_next_level_subtree = get_count(n, ans * 10)`.
    - **Decision:**
      - If `k >= count_nodes_in_next_level_subtree`: This means the $k$-th number is not in the subtree formed by appending a '0' to `ans`. It's one of `ans+1`, `ans+2`, etc., or their subtrees.
        - We "go right": `ans += 1`.
        - We decrement `k` by `count_nodes_in_next_level_subtree` (for the subtree we just skipped)
        - We decrement `k` by `1` (for `ans` itself, because we are moving to `ans+1` which is the next number at this level).
      - Else (`k < count_nodes_in_next_level_subtree`): This means the $k$-th number is in the subtree formed by appending a '0' to `ans`.
        - We "go down": `ans *= 10`.
        - We decrement `k` by `1` (because we effectively chose `ans * 10` as the next prefix, consuming one step).

4.  Return `ans`.

**Trace `n = 13, k = 2` (Revised Algorithm):**

1.  `ans = 1`, `k = 1` (target: 2nd element, 1 step left)
2.  Loop while `k > 0`:
    - `ans = 1`.
    - `count_nodes_in_next_level_subtree = get_count(13, 10)`:
      - `cur = 10`, `next_val = 11`.
      - `cur=10 <= 13`: `count += min(14, 11) - 10 = 1`. (`10` is counted)
      - `cur = 100`, `next_val = 110`.
      - `cur=100 > 13`: Loop ends.
      - `get_count(13, 10)` returns `1`.
    - Is `k >= count_nodes_in_next_level_subtree`? (`1 >= 1` is True)
      - Go right: `ans = 1 + 1 = 2`.
      - `k -= count_nodes_in_next_level_subtree` (`k = 1 - 1 = 0`).
      - `k -= 1` (`k = 0 - 1 = -1`).
    - Loop condition `k > 0` (is `-1 > 0`?) is False. Loop terminates.
3.  Return `ans = 2`.

Still incorrect. The `get_count` calculation is for the _entire subtree_ starting with `prefix`, including numbers that start with `prefix` followed by any digits. The problem is in _what_ we're comparing `k` against.

Let `current` be the current number we are building. Initially `current = 1`. We need to find the $k$-th number.
The main loop will run `k` times, essentially decrementing `k` with each "step" taken towards the target number.

Correct `get_count` logic:
`get_count(n, cur, next)` counts numbers in range `[cur, next-1]` that are `<= n`.
`next_level_start = current * 10`
`next_level_end = (current + 1) * 10`

Let `curr` be the number we are currently _on_. Initially `curr = 1`.
We have `k` steps remaining.

**Finalized Algorithm (Optimized Iterative Traversal):**

1.  Initialize `current_num = 1`.
2.  Decrement `k` by 1 (`k -= 1`). (Because `1` is the first number, and we need to find the `k`-th number _after_ `1` itself).
3.  Loop while `k > 0`:

    - Calculate `count_in_prefix_subtree = get_count_optimized(n, current_num)`.
      - This `get_count_optimized` function counts all numbers _rooted at_ `current_num` (i.e., `current_num`, `current_num0`, `current_num00`, ..., `current_num1`, `current_num10`, ... up to `n`).
      - `get_count_optimized(n, prefix)`:
        - `count = 0`
        - `first = prefix`
        - `last = prefix + 1` (the first number with the _next_ prefix, e.g., if `prefix=1`, `last=2`; if `prefix=10`, `last=11`)
        - While `first <= n`:
          - `count += min(n + 1, last) - first`
          - `first *= 10`
          - `last *= 10`
        - Return `count`.
    - If `k >= count_in_prefix_subtree`: This means the $k$-th number is _not_ in the "subtree" defined by `current_num` and its descendants. It must be a number starting with `current_num + 1` or further.
      - `current_num += 1` (Move to the next sibling, e.g., from `1` to `2`, or `10` to `11`)
      - `k -= count_in_prefix_subtree` (We've skipped all numbers generated by `current_num` and its children).
    - Else (`k < count_in_prefix_subtree`): This means the $k$-th number _is_ in the "subtree" defined by `current_num` and its descendants. We need to go one level deeper.
      - `current_num *= 10` (Go down to `current_num` + '0', e.g., from `1` to `10`)
      - `k -= 1` (We've consumed one step by moving to the next level/number in the sequence).

4.  Return `current_num`.

**Example: `n = 13, k = 2` (Finalized Algorithm)**

1.  `current_num = 1`, `k = 1`
2.  Loop while `k > 0`:
    - `current_num = 1`.
    - `count_in_prefix_subtree = get_count_optimized(13, 1)`:
      - `count = 0`, `first = 1`, `last = 2`
      - `first=1 <= 13`: `count += min(14, 2) - 1 = 1`. (`count = 1`)
        - `first = 10`, `last = 20`
      - `first=10 <= 13`: `count += min(14, 20) - 10 = 4`. (`count = 1 + 4 = 5`)
        - `first = 100`, `last = 200`
      - `first=100 > 13`: Loop ends.
      - `get_count_optimized(13, 1)` returns `5`.
    - Is `k >= count_in_prefix_subtree`? (`1 >= 5` is False).
    - Else (`k < count_in_prefix_subtree`): Go down.
      - `current_num = 1 * 10 = 10`.
      - `k -= 1` (`k = 1 - 1 = 0`).
    - Loop condition `k > 0` (is `0 > 0`?) is False. Loop terminates.
3.  Return `current_num = 10`.

This matches the example output!

**Time Complexity:**
The `k` variable decreases in each iteration.
In the "go down" step (`current_num *= 10`), `current_num` gains a digit. The number of digits in `n` is `O(log10(n))`. This happens at most `O(log10(n))` times.
In the "go right" step (`current_num += 1`), `current_num` changes its last digit. The `get_count_optimized` function also runs in `O(log10(n))` time because its `while` loop runs for each digit position.
Since `k` can be up to `n`, but we are decrementing `k` by the size of subtrees (which can be large) or by 1 (when going down a level), the total number of operations is effectively `O(log K * log N)` or more precisely `O(log^2 N)` in the worst case (if `k` forces many right moves or `n` is very large). Given $N=10^9$, $\log_{10}N \approx 9$. So, $O(9^2)$ is roughly $O(81)$ operations per loop iteration, and the loop runs $O(\log N)$ times. Total time complexity is `O(log^2 N)`. This is very efficient for $N=10^9$.

**Space Complexity:**
`O(1)` extra space (excluding the output if we were to generate it, but we only return one number).

```python
class Solution:
    def findKthNumber(self, n: int, k: int) -> int:

        def get_count_optimized(n_limit: int, prefix: int) -> int:
            """
            Calculates the count of numbers (including prefix) that are in
            the lexicographical subtree rooted at `prefix` and are <= n_limit.

            For example, if n_limit = 13, prefix = 1:
            Counts 1, 10, 11, 12, 13 (total 5).

            If n_limit = 100, prefix = 2:
            Counts 2, 20-29, 200-299... up to 100.
            Actually, it counts 2, 20, 21, ..., 29.
            The loop will iterate:
            cur = 2, next_val = 3: count += min(101, 3) - 2 = 1 (for 2)
            cur = 20, next_val = 30: count += min(101, 30) - 20 = 10 (for 20-29)
            cur = 200, next_val = 300: cur > n_limit, loop ends.
            Total: 1 + 10 = 11.
            """
            count = 0
            # `first` is the starting number of the current level's range
            # `last` is the starting number of the next sibling prefix at the current level,
            # which defines the upper bound for numbers starting with `first`
            # (e.g., if first=1, last=2; if first=10, last=11)
            first = prefix
            last = prefix + 1

            while first <= n_limit:
                # Add the count of numbers in the current block [first, min(n_limit, last - 1)]
                # min(n_limit + 1, last) is used because if last goes over n_limit,
                # we should only count up to n_limit.
                count += min(n_limit + 1, last) - first

                # Move to the next level (deeper in the tree)
                first *= 10
                last *= 10
            return count

        current_num = 1
        k -= 1  # k is 1-indexed, we start from 1, so we need to find the (k-1)-th number *after* 1.

        while k > 0:
            # Calculate how many numbers are in the "subtree" formed by
            # current_num followed by any digits (e.g., if current_num is 1,
            # this counts 10, 11, ..., 19, 100, ..., 199, etc., up to n).
            # This is effectively counting all children of current_num.
            count_in_children_subtree = get_count_optimized(n, current_num * 10)

            if k >= count_in_children_subtree:
                # If k is larger than or equal to the count of numbers in the
                # children's subtree, it means the k-th number is NOT in this
                # subtree. So, we must move to the next sibling (e.g., from 1 to 2, or 10 to 11).
                current_num += 1
                k -= count_in_children_subtree # Subtract the entire skipped subtree
                k -= 1 # Subtract 1 for current_num itself (as we are moving to the next one at this level)
            else:
                # If k is smaller than the count, the k-th number is WITHIN this
                # children's subtree. So, we go deeper (e.g., from 1 to 10).
                current_num *= 10
                k -= 1 # Subtract 1 for moving to the next level/number in the sequence

        return current_num

```

### Test Cases

Let's test the finalized algorithm with the provided examples and some edge cases.

**Example 1:**

- Input: `n = 13, k = 2`
- Expected Output: `10`

**Example 2:**

- Input: `n = 1, k = 1`
- Expected Output: `1`

**Additional Test Cases:**

1.  `n = 10, k = 3`

    - Lexicographical order: `[1, 10, 2, 3, 4, 5, 6, 7, 8, 9]`
    - Expected Output: `2`

2.  `n = 100, k = 15`

    - Order starts: `1, 10, 100, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2, 20, 21, ...`
    - `k=1` (1)
    - `k=2` (10)
    - `k=3` (100)
    - `k=4` (11)
    - ...
    - `k=12` (19)
    - `k=13` (2)
    - `k=14` (20)
    - `k=15` (21)
    - Expected Output: `21`

3.  `n = 9, k = 9`

    - Lexicographical order: `[1, 2, 3, 4, 5, 6, 7, 8, 9]`
    - Expected Output: `9`

4.  `n = 100, k = 1`

    - Expected Output: `1`

5.  `n = 100, k = 100`
    - Expected Output: `99` (last in order)

```python
# Test cases for the Solution class
def run_tests():
    solver = Solution()

    test_cases = [
        (13, 2, 10),
        (1, 1, 1),
        (10, 3, 2),
        (100, 15, 21),
        (9, 9, 9),
        (100, 1, 1),
        (100, 100, 99),
        (12345, 1, 1),
        (12345, 10, 10),
        (12345, 12, 12),
        (12345, 13, 13),
        (12345, 20, 2),
        (12345, 100, 100),
        (12345, 1000, 1000),
        (12345, 10000, 10000),
        (12345, 11000, 11000), # Testing a number that is exactly 11000
        (12345, 12345, 9999), # The largest possible k will be n, testing the very last
        (1000000000, 1, 1),
        (1000000000, 1000000000, 999999999) # Very large n, k
    ]

    for n_val, k_val, expected_output in test_cases:
        actual_output = solver.findKthNumber(n_val, k_val)
        print(f"n={n_val}, k={k_val}")
        print(f"Expected: {expected_output}")
        print(f"Actual:   {actual_output}")
        if actual_output == expected_output:
            print("Status: PASSED\n")
        else:
            print("Status: FAILED (Expected: {expected_output}, Got: {actual_output})\n")

# Run the tests
if __name__ == "__main__":
    run_tests()

```


In [ ]:
class Solution:
    def findKthNumber(self, n: int, k: int) -> int:
        def count_prefix(prefix: int, n: int) -> int:
            """Counts how many numbers in the range [1, n] start with `prefix`."""
            count = 0
            cur, next_val = prefix, prefix + 1
            while cur <= n:
                count += min(n + 1, next_val) - cur
                cur *= 10
                next_val *= 10
            return count

        k -= 1  # Adjust k since we start from 1
        prefix = 1  # Start from '1'
        
        while k > 0:
            count = count_prefix(prefix, n)
            if count <= k:
                k -= count
                prefix += 1  # Move to next prefix
            else:
                k -= 1
                prefix *= 10  # Move deeper in lexicographical order
                
        return prefix

# Edge Cases & Test Cases
test_cases = [
    (13, 2, 10),  # n=13, k=2 -> 10
    (100, 10, 17),  # n=100, k=10 -> 17
    (1, 1, 1),  # Single element case
    (1000000, 10000, 4173),  # Large `n` and `k`
    (500, 100, 200),  # Mid-range n, varying k
]

# Running the test cases
solution = Solution()
for n, k, expected in test_cases:
    result = solution.findKthNumber(n, k)
    print(f"findKthNumber({n}, {k}) = {result}, Expected: {expected}")

**Premature Ejaculation (দ্রুত ফিনিশ হওয়া) - প্রাকৃতিক সমাধানের সম্পূর্ণ গাইড**

### **১. দ্রুত সমাধানের উপায় (তাত্ক্ষণিক কন্ট্রোল)**

✔ **শ্বাস-প্রশ্বাসের ব্যায়াম**

- উত্তেজনা বাড়লে **গভীর শ্বাস নিন (৪ সেকেন্ড), ধরে রাখুন (৪ সেকেন্ড), ছাড়ুন (৪ সেকেন্ড)**
- ৫-১০ বার করুন, উত্তেজনা ৫০% কমে যাবে

✔ **সেনসিটিভিটি কমানোর টিপস**

- কন্ডম ব্যবহার করুন (সেনসেশন কমায়)
- প্রিমিয়াম কন্ডম (Durex Extra Time/Moods) ব্যবহার করলে ২-৩x বেশি সময় পাবেন

### **২. দীর্ঘমেয়াদী সমাধান (১৫-৩০ দিনে ফল)**

#### **ক. হারবাল চিকিৎসা**

🌿 **আশওয়াগন্ধা+দুধ**

- ১ চা চামচ আশওয়াগন্ধা গুঁড়ো + ১ গ্লাস গরম দুধ (রাতে)
- **কারণ:** স্ট্রেস হরমোন কর্টিসল কমায়, স্ট্যামিনা বাড়ায়

🌿 **গোক্ষুর+মধু**

- ১ চামচ গোক্ষুর চূর্ণ + ১ চামচ মধু (সকালে)
- **কারণ:** টেস্টোস্টেরন লেভেল বাড়ায়

#### **খ. বিশেষ এক্সারসাইজ**

💪 **সুপার কিগেল রুটিন**

1. ধীরে টাইট করুন (১০ সেকেন্ড) → ১০ বার
2. দ্রুত টাইট-রিল্যাক্স (১ সেকেন্ড) → ৩০ বার
3. দিনে ৩ সেট (সকাল, দুপুর, রাত)

#### **গ. সেক্সুয়াল ট্রেনিং**

🔥 **এডজিং মেথড**

1. মাস্তুরবেশন শুরু করুন
2. ফিনিশ হওয়ার ৩০ সেকেন্ড আগে থামুন
3. ১ মিনিট রিল্যাক্স করুন
4. আবার শুরু করুন → দিনে ২০ মিনিট প্র্যাকটিস

### **৩. খাদ্য তালিকা (যা খাবেন/এড়াবেন)**

✅ **খাবেন:**

- কলা, তরমুজ, ডার্ক চকোলেট (L-arginine সমৃদ্ধ)
- কুমড়ার বীজ, কাঠবাদাম (জিংক সমৃদ্ধ)

❌ **এড়াবেন:**

- অতিরিক্ত কফি/চা (উত্তেজনা বাড়ায়)
- ফাস্ট ফুড (রক্ত চলাচল কমায়)

### **৪. মনস্তাত্ত্বিক সমাধান**

🧠 **সাইকোলজিক্যাল ট্রিক্স**

- সেক্সের সময় ১০০ থেকে উল্টো গণনা করুন (মাইন্ড ডাইভার্সন)
- Less Focus on Performance, More on Sensation

### **৫. জরুরি টেস্ট (কখন ডাক্তার দেখাবেন)**

⚠️ যদি সাথে এই লক্ষণ থাকে:

- ব্যথা বা জ্বালাপোড়া
- ইরেকশন না হওয়া
- ১ মাস চেষ্টার পরেও না সারলে

### **প্রতিদিনের রুটিন (নমুনা)**

🌅 **সকাল:**

- গোক্ষুর+মধু → ১ চামচ
- কিগেল এক্সারসাইজ → ১ সেট

🌃 **রাত:**

- আশওয়াগন্ধা দুধ → ১ গ্লাস
- এডজিং প্র্যাকটিস → ১৫ মিনিট

**ফল পেতে সময়:**

- ৭ দিনে প্রাথমিক উন্নতি
- ৩০ দিনে ৭০-৮০% উন্নতি

> "ধৈর্য্য ধরুন, প্রতিদিন অনুশীলন করুন। PE সম্পূর্ণ নিয়ন্ত্রণযোগ্য!"

চাইলে আপনার জন্য **কাস্টমাইজড প্ল্যান** তৈরি করে দিতে পারি! 😊  
(বয়স, ওজন ও লাইফস্টাইল জানালে আরো সঠিক পরামর্শ দিতে পারব)


**Premature Ejaculation (PE) কাস্টমাইজড ট্রিটমেন্ট প্ল্যান (বয়স ২৪)**

### **১. দ্রুত ফলাফলের জন্য (৭-১৫ দিনে উন্নতি)**

**সকালের রুটিন:**  
☀️ **৬:৩০ AM** - গরম পানিতে ১ চামচ **মধু + আদার রস** (নার্ভ সেনসিটিভিটি কমায়)  
☀️ **৭:00 AM** - **সুপার কিগেল** (১০ সেকেন্ড হোল্ড + ১০ সেকেন্ড রিল্যাক্স) ৩ সেট

**দুপুরের রুটিন:**  
🌤️ **১:০০ PM** - **কলা + কাঠবাদাম** (পটাশিয়াম ও জিংক)  
🌤️ **২:৩০ PM** - **ডিপ ব্রিদিং এক্সারসাইজ** (৪-৪-৪ পদ্ধতি) ৫ মিনিট

**রাতের রুটিন:**  
🌙 **৯:৩০ PM** - **আশওয়াগন্ধা দুধ** (১ চামচ গুঁড়ো + গরম দুধ)  
🌙 **১০:১৫ PM** - **এডজিং প্র্যাকটিস** (১৫ মিনিট, স্টপ-স্টার্ট মেথড)

---

### **২. সাপ্তাহিক এক্টিভিটি প্ল্যান (৩০ দিনে ৮০% উন্নতি)**

**সোমবার:**

- কিগেল + যোগা (বজ্রাসন ১০ মিনিট)
- সাঁতার ২০ মিনিট

**বুধবার:**

- সাইক্লিং ৩০ মিনিট
- আদা-মধু টনিক

**শুক্রবার:**

- পেলভিক ফ্লোর মাসাজ (নারিকেল তেল দিয়ে)
- এডজিং সেশন

---

### **৩. বিশেষ টিপস (আপনার বয়সের জন্য)**

✔ **টেস্টোস্টেরন বুস্টার:**

- প্রতিদিন **তরমুজ + ডালিম** জুস
- সপ্তাহে ৩ বার **ওমেগা-৩** (চিয়া সিড/সামুদ্রিক মাছ)

✔ **মাইন্ড কন্ট্রোল:**

- সেক্সের সময় **বডি স্ক্যান মেডিটেশন** (মাংসপেশিতে ফোকাস)
- **সেন্সেশন শেয়ারিং টেকনিক** (পার্টনারের সাথে কমিউনিকেশন)

---

### **৪. এভয়েড লিস্ট (যা বর্জন করবেন)**

❌ **রাত জাগা** (ঘুমের সময় টেস্টোস্টেরন তৈরি হয়)  
❌ **টাইট আন্ডারওয়্যার** (অণ্ডকোষের তাপমাত্রা বাড়ায়)  
❌ **স্মোকিং/ভ্যাপিং** (রক্তনালী সংকুচিত করে)

---

### **৫. প্রোগ্রেস ট্র্যাকার**

📅 **সপ্তাহ ১:** কিগেল + এডজিং শুরু  
📅 **সপ্তাহ ২:** শ্বাস-প্রশ্বাসের ব্যায়াম যোগ করুন  
📅 **সপ্তাহ ৩:** পার্টনার সাথে প্র্যাকটিস (যদি সম্ভব হয়)  
📅 **সপ্তাহ ৪:** ৯০% কন্ট্রোল অর্জন

> "এই প্ল্যান ৯০% যুবকের জন্য কাজ করে, শর্ত হলো নিয়মিত অনুশীলন!"

**কোন প্রশ্ন থাকলে জানাতে পারেন!** 😊  
(চাইলে আরো ডিটেইলস দিতে পারি - যেমন কোন দিনে কি খাবেন তার ফুল চার্ট)
